In [ ]:
import pandas as pd
import numpy as np
import geocoder

def get_results_data():
    df = pd.DataFrame()

    for i in range(1, 19):
        year = str(1+(i*101))
        if len(year)<4:
            year = '0'+year
        url = 'https://www.football-data.co.uk/mmz4281/' + year + '/E0.csv'
        df_results = pd.read_csv(url, usecols=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'],  error_bad_lines=False)
        df_results['season'] = year
        df = pd.concat([df, df_results], axis=0)
            
    df.dropna(inplace=True)
    
    return df
 
df_results = get_results_data()


In [ ]:
print(df_results.info())
print(df_results.head())
print(df_results.tail())

In [ ]:
def results_create_datetime(df):
    df['day'] = df['Date'].str.split('/').str[0]
    df['month'] = df['Date'].str.split('/').str[1]
    df['year'] = df['Date'].str.split('/').str[2]
    df['year'] = np.where(df['year'].astype(str).str.len()<3, '20' + df['year'].astype(str), df['year'])
    df['Date'] = pd.to_datetime(df[['year', 'month', 'day']])
    df['PreviousDate'] = df['Date'] - pd.DateOffset(days=1)
    df.drop(['day', 'month', 'year'], axis=1, inplace=True)
    return df

df = results_create_datetime(df_results)
df_results.to_csv('results.csv', index=False)

In [ ]:
df_results = pd.read_csv('results.csv')


In [ ]:
def get_list_of_dates(df):
    df['PreviousDate'] = df['PreviousDate'].dt.date.astype(str)
    dates = list(set(df_results['PreviousDate']))
    dates.sort()
    return dates

dates = get_list_of_dates(df_results)

In [ ]:
def get_elo_data(dates):
    df = pd.DataFrame(columns=['Club', 'Elo', 'Country', 'Level', 'Date'])
    length = len(dates)
    while length>0:
        try:
            for i in dates:
                url = 'http://api.clubelo.com/' + str(i)
                df_elo = pd.read_csv(url, usecols=['Club', 'Elo', 'Country', 'Level'],  error_bad_lines=False)
                df_elo = df_elo[(df_elo['Country']=='ENG') & (df_elo['Level']==1)]
                df_elo['Date'] = i
                df = pd.concat([df, df_elo], sort=True)
                dates.remove(i)
                length = len(dates)
        except:
            pass
    
    df['dup'] = df.duplicated(['Club', 'Date'], keep='first')
    df = df[df['dup']==False]
    df.drop(['dup'], axis=1, inplace=True)
    return df

df_elo = get_elo_data(dates)

In [ ]:
df_elo.info()
df_elo.to_csv('elo.csv', index=False)

In [ ]:
df_results = pd.read_csv('results.csv', dtype={'season': object})
df_elo = pd.read_csv('elo.csv')

In [ ]:
df_elo = df_elo[['Club', 'Date', 'Elo']]

df_elo['Date'] = pd.to_datetime(df_elo['Date']) + pd.DateOffset(days=1)
df_elo['Date'] = df_elo['Date'].dt.date.astype(str)


df_elo.rename(columns = {'Club': 'HomeTeam', 'Elo':'HomeElo'}, inplace=True)
df = df_results.merge(df_elo, on=['HomeTeam', 'Date'], how='left')

df_elo.rename(columns = {'HomeTeam': 'AwayTeam', 'HomeElo':'AwayElo'}, inplace=True)
df = df.merge(df_elo, on=['AwayTeam', 'Date'], how='left')

In [ ]:
teams = list(df['HomeTeam'].unique())
teams.sort()
print(teams)
cities = ['Islington, London', 'Birmingham', 'Birmingham', 'Blackburn', 'Blackpool', 'Bolton', 'Bournemouth', 'Brighton', 'Burnley', 'Cardiff', 
          'Charlton', 'Chelsea', 'Selhurst', 'Derby', 'Liverpool', 'Fulham', 'Huddersfield', 'Hull', 'Ipswich', 'Leeds', 'Leicester', 
          'Liverpool', 'Manchester', 'Manchester', 'Middlesbrough', 'Newcastle', 'Norwich', 'Portsmouth', 'White City, London', 'Reading', 
          'Sheffield', 'Southampton', 'Stoke-On-Trent', 'Sunderland', 'Swansea', 'Tottenham', 'Watford', 'West Bromwich', 
          'Stratford', 'Wigan', 'Wolverhampton']

In [ ]:
def get_coordinates(teams, cities):
    lats = []
    longs = []
    for c in cities:
        query = c +", UK"
        location = geocoder.osm(query)
        lats.append(location.lat)
        longs.append(location.lng)

    df_locations = pd.DataFrame(columns = ['Team', 'Lat', 'Long'])
    df_locations['Team'] = teams
    df_locations['Lat'] = lats
    df_locations['Long'] = longs
    return df_locations

df_locations = get_coordinates(teams, cities)

In [ ]:
df_locations.rename(columns = {'Team': 'HomeTeam', 'Lat':'HomeLat', 'Long':'HomeLong'}, inplace=True)
df = df.merge(df_locations, on = ['HomeTeam'], how='left')

df_locations.rename(columns = {'HomeTeam': 'AwayTeam', 'HomeLat':'AwayLat', 'HomeLong':'AwayLong'}, inplace=True)
df = df.merge(df_locations, on = ['AwayTeam'], how='left')

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(row):

      R = 3959.87433 
      lat1 = row['HomeLat']
      lon1 = row['HomeLong']
      lat2 = row['AwayLat']
      lon2 = row['AwayLong']
      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c


df['Distance'] = df.apply(lambda row: haversine(row), axis=1)

In [ ]:
df['LocalRivals'] = np.where(df['Distance']<15, 1, 0)
df['Distance'] = np.where(df['Distance']==0, 1, df['Distance'])
df['Distance'] = np.log(df['Distance'])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])


df['HomePerf_1'], df['HomePerf_2'], df['HomePerf_3'] = [0, 0, 0]
df['AwayPerf_1'], df['AwayPerf_2'], df['AwayPerf_3'] = [0, 0, 0]
df['HomeWinStreak'], df['AwayWinStreak'] = [0, 0]
df['HomeNonWinStreak'], df['AwayNonWinStreak'] = [0, 0]
df['HomeLosingStreak'], df['AwayLosingStreak'] = [0, 0]
df['HomeNonLosingStreak'], df['AwayNonLosingStreak'] = [0, 0]
df['HomeDrawStreak'], df['AwayDrawStreak'] = [0, 0]
df['HomeLastResult'], df['AwayLastResult'] = ['None', 'None']

In [ ]:
def elo_performance(df, t):
    df['Elo'] = 0
    df['Perf_1'], df['Perf_2'], df['Perf_3'] = [0, 0, 0]
       
    df['Elo'] = np.where(df['HomeTeam']==t, df['HomeElo'], df['Elo'])
    df['Elo'] = np.where(df['AwayTeam']==t, df['AwayElo'], df['Elo'])

    df['Perf_1'] = np.where((df['Team']==t) & (df['Team'].shift()==t) & (df['season'] == df['season'].shift()) , df['Elo'].diff(), df['Perf_1'])
    df['Perf_2'] = np.where((df['Team']==t) & (df['Team'].shift(2)==t) & (df['season'] == df['season'].shift(2)), df['Elo'].diff(2), df['Perf_2'])
    df['Perf_3'] = np.where((df['Team']==t) & (df['Team'].shift(3)==t) & (df['season'] == df['season'].shift(3)), df['Elo'].diff(3), df['Perf_3'])
    
    df['HomePerf_1'] = np.where(df['HomeTeam']==t , df['Perf_1'], df['HomePerf_1'])
    df['HomePerf_2'] = np.where(df['HomeTeam']==t , df['Perf_2'], df['HomePerf_2'])
    df['HomePerf_3'] = np.where(df['HomeTeam']==t , df['Perf_3'], df['HomePerf_3'])
    
    df['AwayPerf_1'] = np.where(df['AwayTeam']==t , df['Perf_1'], df['AwayPerf_1'])
    df['AwayPerf_2'] = np.where(df['AwayTeam']==t , df['Perf_2'], df['AwayPerf_2'])
    df['AwayPerf_3'] = np.where(df['AwayTeam']==t , df['Perf_3'], df['AwayPerf_3'])
    
    df.drop(['Elo', 'Perf_1', 'Perf_2', 'Perf_3'], axis=1, inplace=True)
    return df

In [ ]:
def prev_game(df, t):
    df['date_diff'] = df['Date'].diff()
    df['TeamResult'] = 'None'
    df['LastResult'] = 'None'
    df['TeamResult'] =  np.where((df['Team']==t)  & (df['Team'] == df['HomeTeam']) & (df['FTR']=='H'), 'Win' , df['TeamResult'])
    df['TeamResult'] =  np.where((df['Team']==t)  & (df['Team'] == df['HomeTeam']) & (df['FTR']=='A'), 'Loss' , df['TeamResult'])
    df['TeamResult'] =  np.where((df['Team']==t)  & (df['Team'] == df['AwayTeam']) & (df['FTR']=='A'), 'Win' , df['TeamResult'])
    df['TeamResult'] =  np.where((df['Team']==t)  & (df['Team'] == df['AwayTeam']) & (df['FTR']=='H'), 'Loss' , df['TeamResult'])
    df['TeamResult'] =  np.where((df['Team']==t)   & (df['FTR']=='D'), 'Draw' , df['TeamResult'])
    
    df['LastResult'] = np.where((df['Team'].shift()==t) & (df['date_diff']<'180 days'), df['TeamResult'].shift(), df['LastResult'])
    df['HomeLastResult'] = np.where(df['HomeTeam']==t, df['LastResult'], df['HomeLastResult'])
    df['AwayLastResult'] = np.where(df['AwayTeam']==t, df['LastResult'], df['AwayLastResult'])
    df.drop(['date_diff', 'TeamResult', 'LastResult'], axis=1, inplace=True)
    return df

In [ ]:
def win_streaks(df, t):
    df['date_diff'] = df['Date'].diff()
    
    df['Win'], df['prev'], df['WinStreak'] = [0, 0, 0]
    df['Win'] = np.where((df['Team']==t)  & (df['Team'] == df['HomeTeam']) & (df['FTR']=='H'), 1, df['Win'])
    df['Win'] = np.where((df['Team']==t)  & (df['Team'] == df['AwayTeam']) & (df['FTR']=='A'), 1, df['Win'])
        
    df['prev'] = np.where((df['Team'].shift()==t)  & (df['Win'].shift()==1) & (df['date_diff']<'180 days'), 1,0)
    v = df['prev'].dropna()
    grouper = (v!=v.shift()).cumsum()
    df['WinStreak'] = df.groupby(grouper)['prev'].cumsum()    
    df['HomeWinStreak'] = np.where(df['HomeTeam']==t, df['WinStreak'], df['HomeWinStreak'])
    df['AwayWinStreak'] = np.where(df['AwayTeam']==t, df['WinStreak'], df['AwayWinStreak'])    
    df.drop(['date_diff', 'Win', 'prev', 'WinStreak'], axis=1, inplace=True)
    return df

In [ ]:
def nonwin_streaks(df, t):
    df['date_diff'] = df['Date'].diff()
    
    df['NonWin'], df['prev'], df['NonWinStreak'] = [0, 0, 0]
    df['NonWin'] = np.where((df['Team']==t) & (df['Team'] == df['HomeTeam']) & (df['FTR']!='H'), 1, df['NonWin'])
    df['NonWin'] = np.where((df['Team']==t) & (df['Team'] == df['AwayTeam']) & (df['FTR']!='A'), 1, df['NonWin'])
        
    df['prev'] = np.where((df['Team'].shift()==t) & (df['NonWin'].shift()==1) & (df['date_diff']<'180 days'), 1,0)
    v = df['prev'].dropna()
    grouper = (v!=v.shift()).cumsum()
    df['NonWinStreak'] = df.groupby(grouper)['prev'].cumsum()    
    df['HomeNonWinStreak'] = np.where(df['HomeTeam']==t, df['NonWinStreak'], df['HomeNonWinStreak'])
    df['AwayNonWinStreak'] = np.where(df['AwayTeam']==t, df['NonWinStreak'], df['AwayNonWinStreak'])    
    df.drop(['date_diff', 'NonWin', 'prev', 'NonWinStreak'], axis=1, inplace=True)
    return df

In [ ]:
def losing_streaks(df, t):
    df['date_diff'] = df['Date'].diff()
    
    df['Losing'], df['prev'], df['LosingStreak'] = [0, 0, 0]
    df['Losing'] = np.where((df['Team']==t) & (df['Team'] == df['HomeTeam']) & (df['FTR']=='A'), 1, df['Losing'])
    df['Losing'] = np.where((df['Team']==t) & (df['Team'] == df['AwayTeam']) & (df['FTR']=='H'), 1, df['Losing'])
        
    df['prev'] = np.where((df['Team'].shift()==t) & (df['Losing'].shift()==1) & (df['date_diff']<'180 days'), 1,0)
    v = df['prev'].dropna()
    grouper = (v!=v.shift()).cumsum()
    df['LosingStreak'] = df.groupby(grouper)['prev'].cumsum()    
    df['HomeLosingStreak'] = np.where(df['HomeTeam']==t, df['LosingStreak'], df['HomeLosingStreak'])
    df['AwayLosingStreak'] = np.where(df['AwayTeam']==t, df['LosingStreak'], df['AwayLosingStreak'])    
    df.drop(['date_diff', 'Losing', 'prev', 'LosingStreak'], axis=1, inplace=True)
    return df

In [ ]:
def nonlosing_streaks(df, t):
    df['date_diff'] = df['Date'].diff()
    
    df['NonLosing'], df['prev'], df['NonLosingStreak'] = [0, 0, 0]
    df['NonLosing'] = np.where((df['Team']==t) & (df['Team'] == df['HomeTeam']) & (df['FTR']!='A'), 1, df['NonLosing'])
    df['NonLosing'] = np.where((df['Team']==t) & (df['Team'] == df['AwayTeam']) & (df['FTR']!='H'), 1, df['NonLosing'])
        
    df['prev'] = np.where((df['Team'].shift()==t) & (df['NonLosing'].shift()==1) & (df['date_diff']<'180 days'), 1,0)
    v = df['prev'].dropna()
    grouper = (v!=v.shift()).cumsum()
    df['NonLosingStreak'] = df.groupby(grouper)['prev'].cumsum()    
    df['HomeNonLosingStreak'] = np.where(df['HomeTeam']==t, df['NonLosingStreak'], df['HomeNonLosingStreak'])
    df['AwayNonLosingStreak'] = np.where(df['AwayTeam']==t, df['NonLosingStreak'], df['AwayNonLosingStreak'])    
    df.drop(['date_diff', 'NonLosing', 'prev', 'NonLosingStreak'], axis=1, inplace=True)
    return df

In [ ]:
def draw_streaks(df, t):
    df['date_diff'] = df['Date'].diff()
    
    df['Draw'], df['prev'], df['DrawStreak'] = [0, 0, 0]
    df['Draw'] = np.where((df['Team']==t) & (df['Team'] == df['HomeTeam']) & (df['FTR']=='D'), 1, df['Draw'])
    df['Draw'] = np.where((df['Team']==t) & (df['Team'] == df['AwayTeam']) & (df['FTR']=='D'), 1, df['Draw'])
        
    df['prev'] = np.where((df['Team'].shift()==t) & (df['Draw'].shift()==1) & (df['date_diff']<'180 days'), 1,0)
    v = df['prev'].dropna()
    grouper = (v!=v.shift()).cumsum()
    df['DrawStreak'] = df.groupby(grouper)['prev'].cumsum()    
    df['HomeDrawStreak'] = np.where(df['HomeTeam']==t, df['DrawStreak'], df['HomeDrawStreak'])
    df['AwayDrawStreak'] = np.where(df['AwayTeam']==t, df['DrawStreak'], df['AwayDrawStreak'])    
    df.drop(['date_diff', 'Draw', 'prev', 'DrawStreak'], axis=1, inplace=True)
    return df

In [ ]:
for t in teams:
    df['Team'] = np.where(df['HomeTeam']==t, t, None)
    df['Team'] = np.where(df['AwayTeam']==t, t, df['Team'])
    df.sort_values(by=['Team', 'Date'], inplace=True)

    df = elo_performance(df, t)
    df = prev_game(df, t)
    df = win_streaks(df, t)
    df = nonwin_streaks(df, t)
    df = losing_streaks(df, t)
    df = nonlosing_streaks(df, t)
    df = draw_streaks(df, t)
    
df.drop(['Team'], axis=1, inplace=True)

In [ ]:
df['Day'] = df['Date'].dt.weekday_name
df['Month'] = df['Date'].dt.strftime('%b')
df['Year'] = df['Date'].dt.year

df['Weekend'] = np.where((df['Day']=='Saturday') | (df['Day']=='Sunday'), 1, 0) 

df.sort_values(['Date'], inplace=True)
df['row_count'] = df.groupby('season').cumcount()
df['GameWeek'] = np.floor(df['row_count']/10)+1

In [ ]:
df.dropna(inplace=True)

df.drop(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'season', 'PreviousDate', 
         'HomeLat', 'HomeLong', 'AwayLat', 'AwayLong', 'row_count'], axis=1, inplace=True)
